In [255]:
Lhea Beumer
S4105427

My model integrates the model I made for the previous assignment with a model that includes distraction 
and motivation. It is build on the assumptions that the length of a foreperiod of a previous trial in combination with the length of the foreperiod in the current trial
influences response times. 
For every trial there is a chunk stored that holds both the time of the current fp and the pulses of this current 
fp (which represents the internal clock). Every trial, with exception of the first trial for each subject, a chunk
with the highest activation level is retrieved. From this chunk, the pulses of the time belonging to that fp will be 
transferred back to time, representing the subjects estimate of the current fp in this trial. 
If this estimate (taken from a previous fp) is bigger than the current fp, the subject is prepared and 0.05 s 
will be taken from the max response time of 0.41 s. 
If this estimate (taken from a previous fp) is smaller than the current fp, the subject is unprepared and there
will be a max response time of 0.41 s. 
If this estimate (taken from a previous fp) is bigger than the current fp, is in between the time of 
current foreperiod - 50 ms and the estimate for the next foreperiod, the response time has to be calculated by 
taking the max response time and withdrawing the current foreperiod - their estimate
After every trial, a new chunk will be made that holds the information about the fp in the current trial, so
that it could be retrieved to make an estimate at a later moment again. In other words, every trial a new
chunk will be made.  
The model uses constants for each condition target visibility that are withdrawn from the maximum  
RT, derived from the data. Moreover, I modeled a goal competition between distraction and task goal in that resemble
reward.
I wrote the code that calculates the RTs per trial in a separate cel, so that this can be looped every trial
in the full experiment, the code for which is written in the cel above that. I also included the code I ran
in R studio to visualize the model estimates and the linear mixed-effectmodel.

SyntaxError: invalid syntax (493932489.py, line 1)

In [6]:
from dmchunk import Chunk
from model import Model
import random
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

In [7]:
def noise(s):
    rand = random.uniform(0.001,0.999)
    return s * math.log((1 - rand)/rand)

In [8]:
def pulses_to_time(pulses, t_0 = 0.011, a = 1.1, b = 0.015, add_noise = True):
    
    time = 0
    pulse_duration = t_0
    
    while pulses > 0:
        time = time + pulse_duration
        pulses = pulses - 1
        pulse_duration = a * pulse_duration + add_noise * noise(b * a * pulse_duration)
    
    return time

In [9]:
def time_to_pulses(time, t_0 = 0.011, a = 1.1, b = 0.015, add_noise = True):
    
    pulses = 0
    pulse_duration = t_0
    
    while time >= pulse_duration:
        time = time - pulse_duration
        pulses = pulses + 1
        pulse_duration = a * pulse_duration + add_noise * noise(b * a * pulse_duration)
        
    return pulses

In [10]:
intervals = [0, 0.011, 0.025, 0.1, 0.5, 1, 10]
print("t (s)", "\t", "pulses")
for t in intervals:
    print(t, "\t", time_to_pulses(t))

t (s) 	 pulses
0 	 0
0.011 	 1
0.025 	 2
0.1 	 6
0.5 	 18
1 	 24
10 	 46


In [344]:
class ModelWithMotivation(Model):
    
    #constants tweaked for best model fit   
    da = .7  # distraction activation
    discount = .45 # discount due to motivation drop
    
    #if no reward
    def discount_goal_activation(self):
        self.ga -= self.discount
       
    #if reward 
    def add_goal_activation(self):
        self.ga += self.discount
        
    def __str__(self):
        return "\n=== Model ===\n" \
        "Time: " + str(self.time) + " s \n" \
        "Goal:" + str(self.goal) + "\n" \
        "DM:" + "\n".join([str(c) for c in self.dm]) + "\n" \
        "ga: " + str(self.ga) + "\n" 
    
    def distraction(self):
        return self.da + self.noise(self.s) > self.ga + self.noise(self.s)

In [353]:
# ACT-R defaults:

#create a list of foreperiods
periods=[.3, .6, 9]

#min response time of 280 ms, taken from data
min_response = 0.280 

#50 ms added to RT if not prepared
prepared = 0.05

#add to RT if visibility is low
visib = .02

#activation noise
activation_noise = 0.1

# Experiment timing:

inter_trial_interval = random.uniform(.5, .8)
distraction_mean_time = 0.08 # average distraction time
distraction_variation = 0.05 # variation in distraction (uniform)
focus_loss_probability = 0.2 # probability to lose focus once prepared
focus_latency = 0.2 # if we decide to stay focused, we focus for this amount of time

def distraction_time():
    return random.uniform(distraction_mean_time-distraction_variation,distraction_mean_time+distraction_variation)

def init_model():
    #use the model with motivation
    m = ModelWithMotivation()
    ch1 = Chunk(name = "sr1", slots = {"isa":"foreperiod"})
    ch2 = Chunk(name = "sr2", slots = {"isa":"foreperiod"})
    m.add_encounter(ch1)
    m.add_encounter(ch2)
    m.rt = -2.0
    m.lf = 0.5
    
    return m

In [354]:
import itertools

#code for running the whole experiment with 10 subjects or it takes too long
def experiment(subs=10):
    
    #make dataframe containing the variables for plotting
    results = pd.DataFrame(columns = ['foreperiod', 'sub', 'RT', 'target_visibility', 'reward'])
    
    #loop experiment for all subjects
    for sub in range(subs):
        
        #30 trials per block
        trials= 10
        
        #16 blocks
        blocks = 4
        
        #create a balanced nr of FPs per block resulting in 30 trials per block
        Trials = [.3,.6,.9] * trials 
        
        #create lists with possible reward and visibility situations
        reward = ["yes", "no"]
        visibility = ["hi","lo"]
        
        #per block create a combination of reward and visibility resulting in 16 blocks
        Blocks = list(itertools.product(reward, visibility)) * blocks
        
        #randomize combinations
        random.shuffle(Blocks)
        
        #initialize model
        m = init_model()
        
        m.s = activation_noise
        
        #start at trial 0
        trialNum = 0
        
        goal_activation = False
        
        for block in Blocks:
            
            #randomize foreperiods
            random.shuffle(Trials)
            
            #loop every block
            reward, target_visibility = block
            
            if reward == "no" and goal_activation == False:
                m.discount_goal_activation()
                goal_activation = True
                
            elif reward =="yes" and goal_activation == True:
                m.add_goal_activation()
                goal_activation = False
            
            #loop every trial
            for foreperiod in Trials:

                #calculate the RTs per trial 
                RT = run_one_trial(m, trialNum, foreperiod, target_visibility, reward)
                          
                #increase trial
                trialNum += 1
                
                #append variables of interest to dataframe
                results.loc[len(results)] = [foreperiod, sub, RT, target_visibility, reward]
                
        #make a csv file
        results.to_csv('/Users/lheabeumer/Desktop/Results_model3_1.csv')

In [355]:
def run_one_trial(m, trialNum, foreperiod, target_visibility, reward):
    start_time = m.time 
    done = False
    g = Chunk(name = "goal", slots = {"isa":"goal"})
    m.goal = g
    while not done:

        if m.time - start_time < foreperiod and not m.distraction(): 
            
                # stimulus is not there yet and we are not distracted, so we prepare
                if not "expectation" in m.goal.slots:
                    retrieval = Chunk(name = "new_estimate_pulse", slots = {"isa": "foreperiod"})
                    m.time += 0.05
                    chunk, latency = m.retrieve(retrieval)
                    m.time += latency
                    
                    #try to fill the goal slot if a previous pulse guess exists
                    try:
                        m.goal.slots["expectation"] = pulses_to_time(chunk.slots["previous pulse guess"])
                        
                    #if it doesn't exist, set a high number so there will be a maximum response time
                    except:
                        m.goal.slots["expectation"] = 2
                        
                #stimulus is not there and we are focused so increase time by focus latency       
                else:
                    m.time += focus_latency 
                    
        elif m.time - start_time < foreperiod: #distracted
            # do other things while waiting for the stimulus
            m.time = m.time + distraction_time()
            
            # There is also a probability that we lose focus    
            if "expectation" in m.goal.slots and random.uniform(0.0,1.0) < focus_loss_probability:
                m.goal.slots["expectation"] = 2
                
        # the stimulus has appeared
        else:
            
            # we have a prediction
            if "expectation" in m.goal.slots:
                
                if target_visibility == "lo":
                    response_variable = min_response + visib 
                    
                if target_visibility == "hi":
                    response_variable = min_response    
                    
                #if we were not prepared
                if m.goal.slots["expectation"] > foreperiod:
                    RT = response_variable + prepared

                #if we were prepared
                elif m.goal.slots["expectation"] < foreperiod - prepared:
                    RT = response_variable 

                #we were still preparing
                if m.goal.slots["expectation"] > foreperiod - prepared and m.goal.slots["expectation"] < foreperiod:
                    RT = response_variable + (foreperiod - m.goal.slots["expectation"])
                    
            # we don't have a prediction (we got distracted)
            else:
                RT = min_response + prepared + distraction_time()
    
            #make a new chunk for each new foreperiod estimate
            retrieval = Chunk(name = "new_estimate_pulse"+str([trialNum]), slots = {"isa": "foreperiod", 
                                                            "previous pulse guess": time_to_pulses(foreperiod)})
            #store every new chunk in memory
            m.add_encounter(retrieval)
            
            #increase time by the current foreperiod and trial interval
            m.time += foreperiod + inter_trial_interval
            
            #trial is done
            done = True
       
    #return RT to use in the full experiment
    return RT

In [356]:
experiment()

In [ ]:
#R code used for analysis

#read data
model_CM <- read.csv('Results_model3_1.csv.csv')

#change names of columns so I can use the same code that was provided
colnames(model_CM)[2]  <- "fp"
colnames(model_CM)[3]  <- "sub_id"
colnames(model_CM)[4]  <- "response_time"

#delete a randomly attached column X from the dataframe 
model_CM <- subset(model_CM, select=-X)

#take ms instead of sec, like the data
model_CM$response_time <- 1000 * (model_CM$response_time)

#the code that was provided to computed averages
gave2 <- model_CM  %>% 
	group_by(sub_id, fp, reward, target_visibility)  %>%        # Compute average RT per pp, per condition:
	summarize(RT = mean(response_time))%>% 
	plottab(gv=c('fp','reward','target_visibility'),
            dv='RT', group='sub_id')                            # compute grand avg. w/ plottab()
head(gave2)

#bind the dataframes of the actual data and my model data
all.se <- bind_rows(gave2, gave, .id="model")

#plot 
ggplot(all.se, aes(x=fp, y=RT, linetype=reward,shape=reward, color=model)) +     # ...and plot
	geom_errorbar(aes(ymin=lower,ymax=upper), color='black',size=.5, width=.01) + 
	geom_line(size=.5) + 
	geom_point(size=3) + 
	facet_grid(.~target_visibility, labeller='label_both') + 
	theme_bw(base_size=20) + 
  scale_color_manual(labels = c("model", "data"), values = c("red", "blue")) +
  theme_bw() +
	ylim(275, 410) -> plot_II

fig(10,7)
plot_II

#linear model
m <- lmer(1/response_time ~ fp*reward*target_visibility + (1+fp|sub_id), data=model_CM)
summary(m)
anova(m)
